In [2]:
import earthaccess
from dist_s1_enumerator.mgrs_burst_data import get_mgrs_table
import pandas as pd
from tqdm import tqdm
from pathlib import Path

/mnt/trappist-r0/cmarshak/mambaforge/envs/dist-s1-val/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_mgrs = get_mgrs_table()

In [4]:
earthaccess.login()

# Testing

In [5]:
START_TIME = '2024-12-15'
STOP_TIME = '2025-01-01'

In [6]:
def get_opera_id(query_item) -> str:
    return dict(query_item.__dict__['render_dict'])['meta']['native-id']

def query_dist_hls_cmr(mgrs_tile_id: str, start_time=START_TIME, stop_time=STOP_TIME):
    collection_short_name = 'OPERA_L3_DIST-ALERT-HLS_V1'
    mgrs_bounds = tuple(df_mgrs[df_mgrs.mgrs_tile_id == mgrs_tile_id].total_bounds)
    mgrs_bounds = tuple(float(x) for x in mgrs_bounds)
    datasets_found = earthaccess.search_data(
        short_name=collection_short_name,
        temporal=(start_time, stop_time),
        cloud_hosted=True,
        bounding_box=mgrs_bounds
    )
    datasets_found = [d for d in datasets_found if mgrs_tile_id in get_opera_id(d)]
    return datasets_found

In [7]:
q = query_dist_hls_cmr('18SUJ')


In [8]:
def get_processing_time(query_item: earthaccess.DataGranule) -> pd.Timestamp:
    data = query_item.__dict__['render_dict']
    ts = pd.Timestamp(data['umm']['TemporalExtent']['RangeDateTime']['BeginningDateTime'])
    return ts

In [9]:
#dict(qs_ordered[0].__dict__['render_dict'])

In [10]:
qs_ordered = sorted(q, key=get_processing_time)

In [11]:
# earthaccess.download(q[0], '.')

# Automate

In [46]:
def get_mgrs_tile_id_from_url(url: str):
    zip_path = url.split('/')[-1]
    mgrs_tile_id = zip_path.split('_')[3][1:]
    return mgrs_tile_id

In [47]:
df_0

,zip_url,browse_url,product_request_time,processing_duration,high_confidence_alert_threshold,mgrs_tile_id,post_date_buffer_days,stride_for_norm_param_estimation,n_workers_for_norm_param_estimation,delta_lookback_days_mw,...,model_source,memory_strategy,batch_size_for_norm_param_estimation,post_date,track_number,low_confidence_alert_threshold,n_workers_for_despeckling,device,max_pre_imgs_per_burst_mw,model_compilation
0,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,1025.949,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-08-15,32,2.5,4,best,none,False
1,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,1125.674,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-10-02,32,2.5,4,best,none,False
2,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,809.899,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-05-28,105,2.5,4,best,none,False
3,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,627.532,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-10-07,105,2.5,4,best,none,False
4,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,829.556,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-11-24,105,2.5,4,best,none,False
5,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,846.370,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-02-22,105,2.5,4,best,none,False
6,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,1031.386,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-07-10,32,2.5,4,best,none,False
7,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,1125.263,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-10-14,32,2.5,4,best,none,False
8,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,790.540,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-03-05,105,2.5,4,best,none,False
9,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,759.742,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-08-20,105,2.5,4,best,none,False


In [17]:
df_0 = pd.read_csv('one_val_product.csv')
df_1 = pd.read_csv('val_products_minus_one.csv')
df = pd.concat([df_0, df_1])
df['mgrs_tile_id'] = df.zip_url.map(get_mgrs_tile_id_from_url)
mgrs_tile_ids = df.mgrs_tile_id.unique().tolist()
mgrs_tile_ids[:3]

['50NPL', '45TUK', '21JTF']

In [21]:
len(set(mgrs_tile_ids))

85

In [18]:
def get_most_recent_date_in_timeframe(mgrs_tile_id: str) -> str:
    datasets = query_dist_hls_cmr(mgrs_tile_id)
    if datasets:
        datasets_ordered = sorted(datasets, key=get_processing_time)
        opera_id = dict(datasets_ordered[-1].__dict__['render_dict'])['meta']['native-id']
        out_dir = Path(f'dist_hls/{mgrs_tile_id}/{opera_id}')
        out_dir.mkdir(exist_ok=True, parents=True)
        r = earthaccess.download(datasets_ordered[-1], out_dir)
        return r
    else:
        return ''

In [19]:
out = [get_most_recent_date_in_timeframe(m_id) for m_id in tqdm(mgrs_tile_ids)]

  0%|                 | 0/85 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|█| 19/19 [00:00<00:00

PROCESSING TASKS | :   0%| | 0/19 [00:00<?, ?
PROCESSING TASKS | :   5%| | 1/19 [00:01<00:3
PROCESSING TASKS | :  32%|▎| 6/19 [00:02<00:0
PROCESSING TASKS | :  42%|▍| 8/19 [00:02<00:0
PROCESSING TASKS | :  53%|▌| 10/19 [00:02<00:
PROCESSING TASKS | :  58%|▌| 11/19 [00:02<00:
PROCESSING TASKS | :  68%|▋| 13/19 [00:03<00:
PROCESSING TASKS | :  79%|▊| 15/19 [00:03<00:
PROCESSING TASKS | :  89%|▉| 17/19 [00:03<00:
PROCESSING TASKS | : 100%|█| 19/19 [00:04<00:

COLLECTING RESULTS | : 100%|█| 19/19 [00:00<0
  1%|         | 1/85 [00:05<08:19,  5.95s/it]
QUEUEING TASKS | : 100%|█| 19/19 [00:00<00:00

PROCESSING TASKS | :   0%| | 0/19 [00:00<?, ?
PROCESSING TASKS | :   5%| | 1/19 [00:01<00:1
PROCESSING TASKS | :  11%| | 2/19 [00:01<00:1
PROCESSING TASKS | :  21%|▏| 4/19 [00:01<00:0
PROCESSING TASKS | :  53%|▌| 10/19 [00:02<00:
PROCESSING TASKS | :  63%|▋| 12/19 [00:02<00:
PROCESSING TASKS | :  84%|▊| 16

In [25]:
len(out)

85

In [39]:
harder_mgrs_tile_ids = []
for mgrs_tile_id, l_out in zip(mgrs_tile_ids, out):
    if not l_out:
        print(mgrs_tile_id)
        harder_mgrs_tile_ids.append(mgrs_tile_id)

38ULE
41UPB
12UUD
35VLF
32VMQ
52VCH
37VDH
52UCF


In [33]:
len(harder_mgrs_tile_ids)

8

In [44]:
def get_most_recent_date_in_timeframe_harder(mgrs_tile_id: str) -> str:
    datasets = query_dist_hls_cmr(mgrs_tile_id, start_time='2024-09-01')
    if datasets:
        datasets_ordered = sorted(datasets, key=get_processing_time)
        opera_id = dict(datasets_ordered[-1].__dict__['render_dict'])['meta']['native-id']
        out_dir = Path(f'dist_hls/{mgrs_tile_id}/{opera_id}')
        out_dir.mkdir(exist_ok=True, parents=True)
        r = earthaccess.download(datasets_ordered[-1], out_dir)
        return r
    else:
        return ''

In [45]:
list(map(get_most_recent_date_in_timeframe_harder,harder_mgrs_tile_ids ))

QUEUEING TASKS | : 100%|█| 19/19 [00:00<00:00
PROCESSING TASKS | : 100%|█| 19/19 [00:00<00:
COLLECTING RESULTS | : 100%|█| 19/19 [00:00<0
QUEUEING TASKS | : 100%|█| 19/19 [00:00<00:00
PROCESSING TASKS | : 100%|█| 19/19 [00:00<00:
COLLECTING RESULTS | : 100%|█| 19/19 [00:00<0
QUEUEING TASKS | : 100%|█| 19/19 [00:00<00:00
PROCESSING TASKS | : 100%|█| 19/19 [00:00<00:
COLLECTING RESULTS | : 100%|█| 19/19 [00:00<0
QUEUEING TASKS | : 100%|█| 19/19 [00:00<00:00
PROCESSING TASKS | : 100%|█| 19/19 [00:00<00:
COLLECTING RESULTS | : 100%|█| 19/19 [00:00<0
QUEUEING TASKS | : 100%|█| 19/19 [00:00<00:00
PROCESSING TASKS | : 100%|█| 19/19 [00:04<00:
COLLECTING RESULTS | : 100%|█| 19/19 [00:00<0
QUEUEING TASKS | : 100%|█| 19/19 [00:00<00:00
PROCESSING TASKS | : 100%|█| 19/19 [00:00<00:
COLLECTING RESULTS | : 100%|█| 19/19 [00:00<0
QUEUEING TASKS | : 100%|█| 19/19 [00:00<00:00
PROCESSING TASKS | : 100%|█| 19/19 [00:00<00:
COLLECTING RESULTS | : 100%|█| 19/19 [00:00<0
QUEUEING TASKS | : 100%|█| 19/19 [

[['dist_hls/38ULE/OPERA_L3_DIST-ALERT-HLS_T38ULE_20241122T081149Z_20241202T173851Z_S2B_30_v1/OPERA_L3_DIST-ALERT-HLS_T38ULE_20241122T081149Z_20241202T173851Z_S2B_30_v1_VEG-DIST-STATUS.tif',
  'dist_hls/38ULE/OPERA_L3_DIST-ALERT-HLS_T38ULE_20241122T081149Z_20241202T173851Z_S2B_30_v1/OPERA_L3_DIST-ALERT-HLS_T38ULE_20241122T081149Z_20241202T173851Z_S2B_30_v1_VEG-IND.tif',
  'dist_hls/38ULE/OPERA_L3_DIST-ALERT-HLS_T38ULE_20241122T081149Z_20241202T173851Z_S2B_30_v1/OPERA_L3_DIST-ALERT-HLS_T38ULE_20241122T081149Z_20241202T173851Z_S2B_30_v1_VEG-ANOM.tif',
  'dist_hls/38ULE/OPERA_L3_DIST-ALERT-HLS_T38ULE_20241122T081149Z_20241202T173851Z_S2B_30_v1/OPERA_L3_DIST-ALERT-HLS_T38ULE_20241122T081149Z_20241202T173851Z_S2B_30_v1_VEG-HIST.tif',
  'dist_hls/38ULE/OPERA_L3_DIST-ALERT-HLS_T38ULE_20241122T081149Z_20241202T173851Z_S2B_30_v1/OPERA_L3_DIST-ALERT-HLS_T38ULE_20241122T081149Z_20241202T173851Z_S2B_30_v1_VEG-ANOM-MAX.tif',
  'dist_hls/38ULE/OPERA_L3_DIST-ALERT-HLS_T38ULE_20241122T081149Z_20241202T